In [8]:
import random
import numpy as np
from fractions import Fraction

In [5]:
def is_feasible(solution, A, b, eps):
    b_est = A@solution
    sse = sum([(x-y)**2 for x,y in zip(b, b_est)])

    return sse < eps

In [16]:
small_data_dim = [(5,5), (10,5), (10,10),(15,5),(15,10)]
small_data_intervals = [(-10, 10), (-20, 20), (-30, 30), (-40, 40), (-50, 50)]
big_data_dim = [(30,30), (40,30),(100,50), (100,100), (100,150)]
big_data_intervals = [(-150, 150), (-200,200), (-250,250), (-300,300), (-350,350)]

In [3]:
# Define matrix dimensions
m, n = 5, 10  # m rows, n columns


A = np.random.randint(-10, 10, size=(m, n))
b = np.random.randint(-10, 10, size=m)

# Compute real-valued solution
x_real = np.linalg.lstsq(A, b, rcond=None)[0]

# Convert the real solution to rational numbers
x_rational = [Fraction(xi).limit_denominator() for xi in x_real]

# Print the results
print("Matrix A:\n", A)
print("Vector b:\n", b)
print("Real solution x:\n", x_real)
print("Rational solution x:\n", x_rational)


Matrix A:
 [[  6   2  -8   0   4   3  -2   3 -10   8]
 [ -3  -7   7   4  -1   8  -5  -8   2  -5]
 [  8  -3   5  -8   9  -6  -3 -10   2  -7]
 [ -5  -3  -4  -4   1  -5 -10   8   6   8]
 [  8  -8  -7   3  -5  -7   8  -6  -4   9]]
Vector b:
 [ 4  1  7 -1  0]
Real solution x:
 [ 0.2298624  -0.03412642 -0.00903547 -0.13089183  0.27489601  0.05084242
 -0.18123707 -0.15354721 -0.16126901 -0.01872991]
Rational solution x:
 [Fraction(228677, 994843), Fraction(-33212, 973205), Fraction(-8737, 966967), Fraction(-69505, 531011), Fraction(2247, 8174), Fraction(43753, 860561), Fraction(-28381, 156596), Fraction(-78152, 508977), Fraction(-148198, 918949), Fraction(-15975, 852914)]


In [11]:
x_est = [float(xi) for xi in x_rational]
x_est[2] = 0
x_est[9] = 0
b_est = A@x_est
is_feasible(x_est, A, b, 0.05)

np.True_

In [15]:
from itertools import combinations
def brute_force_mrvls(A, b):
    n = A.shape[1] #broj kolona
    best_subset = None
    min_nonzero_vars = n + 1  

    for r in range(1, n + 1):
        # za svaki moguci broj izabranih kolona biramo podmatrice izborom kolona
        for cols in combinations(range(n), r):
            A_sub = A[:, cols]  # uzimamo podmatricu za izabrane kolone
            try:
                # proveravamo da li je sistem resiv
                x_sub = np.linalg.lstsq(A_sub, b, rcond=None)[0]
                # proveravamo da li je resenje dovoljno dobro
                if is_feasible(x_sub, A_sub, b, 0.05):
                    if len(cols) < min_nonzero_vars:
                        best_subset = cols
                        min_nonzero_vars = len(cols)
            except np.linalg.LinAlgError:
                # ako je sistem neresiv, preskociti podmatricu
                continue

    if best_subset is not None:
        x_optimal = np.zeros(n)
        x_sub = np.linalg.lstsq(A[:, best_subset], b, rcond=None)[0]
        x_optimal[list(best_subset)] = x_sub
        return (x_optimal, min_nonzero_vars)
    else:
        return None


x, min_non_zero_vars = brute_force_mrvls(A, b)
print(x)
print(min_non_zero_vars)


[2.   2.   6.75 0.   0.   0.   0.   0.   0.   5.25]
4


In [22]:
for i,((m,n),(a,b)) in enumerate(zip(small_data_dim, small_data_intervals)):
    A = np.random.randint(a, b, size=(m,n))
    b = np.random.randint(a,b, size=m)
    np.save(f'data/small_data/{i}_A', A)
    np.save(f'data/small_data/{i}_b', b)
    

In [23]:
for i,((m,n),(a,b)) in enumerate(zip(big_data_dim, big_data_intervals)):
    A = np.random.randint(a, b, size=(m,n))
    b = np.random.randint(a,b, size=m)
    np.save(f'data/big_data/{i}_A', A)
    np.save(f'data/big_data/{i}_b', b)